### Web scrabed using the coded present on the link below
https://maoviola.medium.com/a-complete-guide-to-web-scraping-linkedin-job-postings-ad290fcaa97f

### Used the link below to write/append existing dataframe to an existing google sheet
https://medium.com/@jb.ranchana/write-and-append-dataframes-to-google-sheets-in-python-f62479460cf0

### link to the google sheet where we will be tracking job postings.
https://docs.google.com/spreadsheets/d/1RnwjV0Qd_OdMWSRLoVsUOs_mKn5DEgMDOwD0Hhsc0Kg/edit#gid=0

### Read and Write data in GS 
https://aryanirani123.medium.com/read-and-write-data-in-google-sheets-using-python-and-the-google-sheets-api-6e206a242f20

### Instructions on how to create a CRON
https://www.jcchouinard.com/python-automation-with-cron-on-mac/#What_You_Will_Need

In [1]:
# All imports
from selenium import webdriver
import time
import pandas as pd
import os

In [2]:
url="https://www.linkedin.com/jobs/search/?currentJobId=3170092398&geoId=105149290&keywords=coop%20data%20analyst&location=Ontario%2C%20Canada&refresh=true"

In [3]:

os.chdir('/Users/tony/Desktop/')

In [5]:
# headless mode
# options = webdriver.ChromeOptions()
# options.add_argument("headless")
# wd = webdriver.Chrome(chrome_options=options)
# wd.get(url)

# window popup
wd = webdriver.Chrome()
wd.get(url)

In [6]:
# lists no. of job posting
no_of_jobs = int(wd.find_element("css selector", "h1 > span").get_attribute("innerText"))


In [7]:
# scrolling for you
i = 2
while i <= int(no_of_jobs/25)+1: 
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        wd.find_element_by_xpath("/html/body/main/div/section/button").click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

In [8]:
job_lists = wd.find_element('class name','jobs-search__results-list')
jobs = job_lists.find_elements('tag name','li') # return a list

In [9]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []
for job in jobs:
    job_id0 = job.get_attribute("data-id")
    job_id.append(job_id0)

    job_title0 = job.find_element('css selector','h3').get_attribute('innerText')
    job_title.append(job_title0)

    company_name0 = job.find_element('css selector','h4').get_attribute('innerText')
    company_name.append(company_name0)

    location0 = job.find_element('css selector',"[class='job-search-card__location']").get_attribute('innerText')
    location.append(location0)
   

    date0 = job.find_element('css selector','div>div>time').get_attribute('datetime')
    date.append(date0)

    job_link0 = job.find_element('css selector','a').get_attribute('href')
    job_link.append(job_link0)

In [10]:
df=pd.DataFrame({'Title':job_title,'Company_name':company_name,'Location':location,'Date':date,'Job Link':job_link})

In [11]:

import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

scopes = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

In [12]:
credentials = Credentials.from_service_account_file(filename='lucid-destiny-344212-cbbebad672a7.json',scopes=scopes)

In [13]:
gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)


In [14]:
# # open a google sheet
# gs = gc.open_by_url('https://docs.google.com/spreadsheets/d/1RnwjV0Qd_OdMWSRLoVsUOs_mKn5DEgMDOwD0Hhsc0Kg/edit#gid=0')
# # select a work sheet from its name
# worksheet1 = gs.worksheet('Sheet1')

# # write to dataframe (first load)
# worksheet1.clear()
# set_with_dataframe(worksheet=worksheet1, dataframe=df, include_index=False,
# include_column_header=True)


In [ ]:
open_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1RnwjV0Qd_OdMWSRLoVsUOs_mKn5DEgMDOwD0Hhsc0Kg/edit#gid=0').sheet1   
data = open_sheet.get_all_records() 
df2=pd.DataFrame(data)

In [ ]:
df3=pd.concat([df2,df])

In [ ]:
df3

In [ ]:
df3.drop_duplicates(keep='first',inplace=True, subset=['Title','Company_name','Location','Date'])

In [ ]:
# worksheet1=sheet.worksheet('Sheet1')
# worksheet1.clear()
#worksheet1 = open_sheet.worksheet('Sheet1')
open_sheet.clear()
set_with_dataframe(worksheet=open_sheet, dataframe=df3, include_index=False,include_column_header=True)